In [63]:
import requests
import pandas as pd
import duckdb
import json
import datamart_profiler
import io
id="datamart.socrata.data-cityofnewyork-us.mebz-ditc"
filename=id.replace(".","_").replace("-","_")+".parquet"
resp=requests.get('https://auctus.vida-nyu.org/api/v1/download/'+id)

In [64]:
df = pd.read_csv(io.BytesIO(resp.content))
df

,borough,communityboard,councildistrict,multipolygon,objectid,parkdistrict,sector
0,Q,404,212529,MULTIPOLYGON (((-73.85135102815293 40.74311905...,186,Q-04,Q-S3.4
1,Q,403481,2125,MULTIPOLYGON (((-73.86968683172124 40.77176551...,185,Q-03,Q-S3.4
2,Q,402,2630,MULTIPOLYGON (((-73.94919060193513 40.75442950...,184,Q-02,Q-S1.2
3,Q,401,2226,MULTIPOLYGON (((-73.90401801027687 40.78973110...,183,Q-01,Q-S1.2
4,B,355,353933,MULTIPOLYGON (((-73.97633386119317 40.66521360...,182,B-19,B-S14.19
...,...,...,...,...,...,...,...
70,M,112,710,MULTIPOLYGON (((-73.92299018419125 40.85820362...,213,M-12A,M-S12.12A
71,M,111,489,MULTIPOLYGON (((-73.92251742210699 40.85782010...,212,M-11,M-S10.11
72,M,111,8,MULTIPOLYGON (((-73.92464543028431 40.80200323...,211,M-11R,M-S11R
73,X,212227,111215,MULTIPOLYGON (((-73.87812575244912 40.88215368...,210,X-14,X-S11.12.14


In [ ]:
data=resp.content.decode()
list=[x.split(',') for x in data.split('\n')]
print(list[1])
df=pd.DataFrame(list[1:],columns=list[0])


In [ ]:
df.columns

In [ ]:
profiles=datamart_profiler.process_dataset(df)
for c in profiles['columns']:
    dtypes=[]
    dtypes.append(c['structural_type'].split('/')[-1])
    for s in c['semantic_types']:
        dtypes.append(s.split('/')[-1])
    print(c['name'],dtypes)

In [13]:
df.to_parquet(filename)
df.to_csv('test_file.csv')  

In [14]:
con=duckdb.connect("testdb.db")
con.sql(" describe select * from "+filename).fetchdf()

,column_name,column_type,null,key,default,extra
0,Country,VARCHAR,YES,NaN,NaN,NaN
1,Total cases,VARCHAR,YES,NaN,NaN,NaN
2,New cases,VARCHAR,YES,NaN,NaN,NaN
3,Total deaths,VARCHAR,YES,NaN,NaN,NaN
4,New deaths,VARCHAR,YES,NaN,NaN,NaN
5,Total recovered,VARCHAR,YES,NaN,NaN,NaN
6,Active cases,VARCHAR,YES,NaN,NaN,NaN
7,Critical cases,VARCHAR,YES,NaN,NaN,NaN
8,Cases/1M pop,VARCHAR,YES,NaN,NaN,NaN
9,Lon,VARCHAR,YES,NaN,NaN,NaN


In [16]:
df_cpy=df.copy()
df_cpy
profiles=datamart_profiler.process_dataset(df_cpy)
for c in profiles['columns']:
    dtypes=[]
    dtypes.append(c['structural_type'].split('/')[-1])
    for s in c['semantic_types']:
        dtypes.append(s.split('/')[-1])
    try:
        if "DateTime" in dtypes:
            df_cpy[c['name']] = pd.to_datetime(df_cpy[c['name']], format='%Y-%m-%d')
        elif "Integer" in dtypes:
            df_cpy[c['name']]=df_cpy[c['name']].fillna(0).astype(float).astype(int)
        elif "Float" in dtypes:
            df_cpy[c['name']]=df_cpy[c['name']].fillna(0).astype(float)
        elif "Text" in dtypes:
            df_cpy[c['name']]=df_cpy[c['name']].fillna("null").astype(str)
    except:
        pass


/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/auctus/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the va

In [25]:
df

,Country,Total cases,New cases,Total deaths,New deaths,Total recovered,Active cases,Critical cases,Cases/1M pop,Lon,Lat\r
0,China,81054.0,46.0,3261.0,6.0,72440.0,5353.0,1845.0,56.0,104.999927,35.000074\r
1,Italy,59138.0,5560.0,5476.0,651.0,7024.0,46638.0,3000.0,978.0,12.674297,42.6384261\r
2,USA,38757.0,14550.0,400.0,98.0,178.0,38179.0,795.0,117.0,-100.4458825,39.7837304\r
3,Spain,28603.0,3107.0,1756.0,375.0,2125.0,24722.0,1785.0,612.0,-4.8380649,39.3262345\r
4,Germany,24806.0,2442.0,93.0,9.0,266.0,24447.0,2.0,296.0,10.4234469,51.0834196\r
...,...,...,...,...,...,...,...,...,...,...,...
187,Somalia,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.06,49.083416,8.3676771\r
188,Syria,1.0,1.0,NaN,NaN,NaN,1.0,NaN,0.06,39.0494106,34.6401861\r
189,Timor-Leste,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.8,125.8375756,-8.5151979\r
190,Uganda,1.0,NaN,NaN,NaN,NaN,1.0,NaN,0.02,32.2166578,1.5333554\r


In [49]:
import duckdb
con=duckdb.connect("play.db")
con.sql("install 'httpfs';")
con.sql("load 'httpfs';")
con.sql("SET s3_url_style='path';")
con.sql(" SET s3_endpoint='localhost:8050'")
con.sql("SET s3_use_ssl=false;")
con.sql("SET s3_access_key_id='devkey';")
con.sql("SET s3_secret_access_key='devpassword';")
con.sql("select * from read_parquet('s3://auctus-bucket/df.parquet');")

['TIMESTAMP', 'BIGINT', 'BIGINT']